In [14]:

import numpy as np
import tensorflow as tf
from bidict import bidict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from pymongo import MongoClient
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.metrics import confusion_matrix


In [15]:
# Création d'un dictionnaire bidirectionnel pour encoder les chiffres de 0 à 9.
ENCODER = bidict({
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, 
    '6': 6, '7': 7, '8': 8, '9': 9,
    
})

In [16]:
# Utilisation de la bibliothèque numpy pour charger le jeu de données MNIST à partir d'un fichier CSV.
data = np.loadtxt('../data/mnist_train.csv', delimiter=',')
# Séparation des étiquettes et des images. 
# Les étiquettes sont dans la première colonne, tandis que le reste des colonnes contient les pixels des images.
labels = data[:, 0]
images = data[:, 1:]

In [17]:

# Normalisation des images. Les valeurs des pixels sont divisées par 255 pour obtenir des valeurs comprises entre 0 et 1.

images = images / 255.0


In [18]:
# Remodelage des images pour qu'elles aient la forme (28, 28, 1), 
images = images.reshape(-1, 28, 28, 1)


In [19]:
# Encodage des étiquettes en catégories. C'est une étape nécessaire pour l'entraînement avec la fonction de perte 'categorical_crossentropy'.
labels_encoded = to_categorical(labels)
labels_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [20]:
# Division du jeu de données en ensembles d'entraînement et de validation.
X_train, X_val, y_train, y_val = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)


In [21]:
# Création d'un modèle de réseau neuronal convolutif à l'aide de Keras.
model = Sequential([
    # Première couche convolutive avec 32 filtres de taille (3,3) et une fonction d'activation ReLU.
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    # Couche de mise en commun pour réduire la taille des cartes de caractéristiques.
    MaxPooling2D(2, 2),
    # Deuxième couche convolutive avec 64 filtres.
    Conv2D(64, (3,3), activation='relu'),
    # Deuxième couche de mise en commun.
    MaxPooling2D(2, 2),
    # Aplatir les cartes de caractéristiques pour les connecter à des couches entièrement connectées.
    Flatten(),
    # Couche entièrement connectée avec 128 neurones.
    Dense(128, activation='relu'),
    # Couche de sortie avec 10 neurones (pour les 10 classes de chiffres) et une activation softmax.
    Dense(10, activation='softmax')
])

In [22]:
# Compilation du modèle avec l'optimiseur Adam, la fonction de perte 'categorical_crossentropy' et la métrique 'accuracy'.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
# Entraînement du modèle sur les données d'entraînement, avec validation sur les données de validation.
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)



Epoch 1/10
1500/1500 [==============================] - 45s 28ms/step - loss: 0.1505 - accuracy: 0.9537 - val_loss: 0.0657 - val_accuracy: 0.9791
Epoch 2/10
1500/1500 [==============================] - 33s 22ms/step - loss: 0.0484 - accuracy: 0.9843 - val_loss: 0.0455 - val_accuracy: 0.9859
Epoch 3/10
1500/1500 [==============================] - 33s 22ms/step - loss: 0.0317 - accuracy: 0.9896 - val_loss: 0.0448 - val_accuracy: 0.9866
Epoch 4/10
1500/1500 [==============================] - 32s 22ms/step - loss: 0.0223 - accuracy: 0.9929 - val_loss: 0.0452 - val_accuracy: 0.9859
Epoch 5/10
1500/1500 [==============================] - 42s 28ms/step - loss: 0.0168 - accuracy: 0.9947 - val_loss: 0.0400 - val_accuracy: 0.9881
Epoch 6/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0127 - accuracy: 0.9961 - val_loss: 0.0429 - val_accuracy: 0.9883
Epoch 7/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0101 - accuracy: 0.9967 - val_loss: 0.0417 -

In [24]:
# Sauvegarde du modèle entraîné pour une utilisation ultérieure.
model.save('../chiffre.model')

INFO:tensorflow:Assets written to: ../chiffre.model\assets


INFO:tensorflow:Assets written to: ../chiffre.model\assets
